In [ ]:
[root@ip-172-31-8-221 docker-image]# ll
total 24
-r-------- 1 root root 1679 Jul  1 13:34 ansible.pem
-rw------- 1 root root   57 Jul  1 21:42 cred.vars
-rw-r--r-- 1 root root 3389 Jul  1 23:23 docker-image.yml
-rw-r--r-- 1 root root   18 Jul  1 23:37 docker.vars
-rw-r--r-- 1 root root  228 Jul  1 22:41 hosts
-rw-r--r-- 1 root root   93 Jul  1 18:32 image.vars
[root@ip-172-31-8-221 docker-image]# cat docker.vars 
docker_port: 8081
[root@ip-172-31-8-221 docker-image]# cat image.vars 
--- 
git_url: "https://github.com/jinumona/devops-flask.git"
git_dir: "/home/ec2-user/image"
[root@ip-172-31-8-221 docker-image]# cat cred.vars 
---
hub_username: jinumona
hub_password: 

In [ ]:
- name: "Debug git_status from host amazon-image-server"
      debug:
        var: hostvars['172.31.14.65']['git_status']
            
###########
TASK [Debug git_status from host amazon-image-server] *****************************************************************************************************
ok: [172.31.8.225] => {
    "hostvars['172.31.14.65']['git_status']": {
        "after": "8db1708be153c3b60885c1cc6ce41832ff4e2ad7", 
        "before": "8db1708be153c3b60885c1cc6ce41832ff4e2ad7", 
        "changed": false, 
        "failed": false, 
        "remote_url_changed": false
    }
}


                

In [ ]:
 - name: "Debug git_status from host amazon-image-server"
      debug:
        var: hostvars['172.31.14.65']['git_status'].changed
            
            
##########
TASK [Debug git_status from host amazon-image-server] *****************************************************************************************************
ok: [172.31.8.225] => {
    "hostvars['172.31.14.65']['git_status'].changed": false
}

            

In [ ]:
# cat docker-image.yml 
---
- name: "Creating docker Image on Amazon Linux"
  hosts: amazon-image-server
  become: yes
  vars_files:
    - image.vars 
    - cred.vars

  tasks:
    - name: "Installing Docker,pip,git on Amazon Linux"
      yum:
        name: 
          - docker
          - python-pip
          - git
        state: present
      register: pkg_status



    - name: "Installing python-docker connector/module"
      pip:
        name: 
          - docker-py
        state: present

    - name: "Adding ec2-user to docker group"
      user:
        append: yes
        groups: docker
        name: ec2-user

    - name: "Restarting/enabling docker"
      when: pkg_status.changed == true 
      service:
        name: docker 
        state: restarted
        enabled: yes


    - name: "cloning Git Docker Files"
      git:
        dest: "{{ git_dir }}"
        repo: "{{ git_url }}"
      register: git_status

    - name: "Debug git"
      debug:
        var: git_status 
    

    - name: "Building Docker Image"
      when: git_status.changed == true
      community.docker.docker_image:
        name: jinumona/devops-flaskapp
        build:
          path: "{{ git_dir }}"
        source: build 
        state: present
        tag: "{{ item }}"
        force_source: yes
        force_tag: yes
      with_items:
        - "{{ git_status.after }}"
        -  latest
      register: image_build_status

    - name: "Debug image_build_status"
      debug:
        var: image_build_status

    - name: "Logging to Dockerhub...."
      when: git_status.changed == true
      community.docker.docker_login:
        password: "{{ hub_password }}"
        username: "{{ hub_username }}"


    - name: "Pushing the latest version changed Dockerimage to Hub"
      when: git_status.changed == true
      community.docker.docker_image:
        name: "jinumona/devops-flaskapp:{{ item }}"
        push: yes
        source: local
        force_tag: yes
      with_items: 
        - "{{ git_status.after }}"
        - latest


    - name: "Logout from Hub"
      when: git_status.changed == true
      community.docker.docker_login:
        state: absent


    
    - name: "removing the created local images"
      when: git_status.changed == true
      community.docker.docker_image:
        name: "jinumona/devops-flaskapp:{{ item }}"
        state: absent
      with_items: 
        - "{{ git_status.after }}"
        - latest




- name: "Deploying version chnaged application on to docker container of dev-server"
  hosts: dev-server
  become: yes
  vars_files:
    - docker.vars 

  tasks:
    - name: "Installing Docker on Amazon Linux dev-server"
      yum:
        name: 
          - docker
          - python-pip
        state: present
      register: docker_status

    - name: "python docker connector/module"
      pip:
        name: 
          - docker-py
        state: present

    - name: "Appending ec2-user to docker group"
      user:
        append: yes
        groups: docker
        name: ec2-user

    - name: "Restarting/enabling docker"
      when: docker_status.changed == true 
      service:
        name: docker 
        state: restarted
        enabled: yes
  

    - name: "Creating container at dev-server"
      community.docker.docker_container:
        state: started
        detach: yes
        name: "myflaskcntr{{ docker_port }}"
        image: jinumona/devops-flaskapp:latest
        pull: yes
        restart_policy: always
        published_ports:
          - "{{ docker_port }}:5000"
      register: cntr_info

    - name: "Debug container"
      debug:
        msg: "Container Hostname:{{ cntr_info.container.Config.Hostname }} Changed:{{ cntr_info.changed }} created at:{{ cntr_info.container.Created }} Restarting state:{{ cntr_info.container.State.Restarting }}"
        
    
    - name: "Debug git_status from host amazon-image-server"
      debug:
        var: hostvars['172.31.14.65']['git_status']


In [ ]:
$ ansible-playbook -i hosts docker-image.yml 

In [ ]:

[WARNING]: Invalid characters were found in group names but not replaced, use -vvvv to see details

PLAY [Creating docker Image on Amazon Linux] **************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.14.65 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.14.65]

TASK [Installing Docker,pip,git on Amazon Linux] **********************************************************************************************************
ok: [172.31.14.65]

TASK [Installing python-docker connector/module] **********************************************************************************************************
ok: [172.31.14.65]

TASK [Adding ec2-user to docker group] ********************************************************************************************************************
ok: [172.31.14.65]

TASK [Restarting/enabling docker] *************************************************************************************************************************
skipping: [172.31.14.65]

TASK [cloning Git Docker Files] ***************************************************************************************************************************
ok: [172.31.14.65]

TASK [Debug git] ******************************************************************************************************************************************
ok: [172.31.14.65] => {
    "git_status": {
        "after": "8db1708be153c3b60885c1cc6ce41832ff4e2ad7", 
        "before": "8db1708be153c3b60885c1cc6ce41832ff4e2ad7", 
        "changed": false, 
        "failed": false, 
        "remote_url_changed": false
    }
}

TASK [Building Docker Image] ******************************************************************************************************************************
skipping: [172.31.14.65] => (item=8db1708be153c3b60885c1cc6ce41832ff4e2ad7) 
skipping: [172.31.14.65] => (item=latest) 

TASK [Debug image_build_status] ***************************************************************************************************************************
ok: [172.31.14.65] => {
    "image_build_status": {
        "changed": false, 
        "msg": "All items completed", 
        "results": [
            {
                "ansible_loop_var": "item", 
                "changed": false, 
                "item": "8db1708be153c3b60885c1cc6ce41832ff4e2ad7", 
                "skip_reason": "Conditional result was False", 
                "skipped": true
            }, 
            {
                "ansible_loop_var": "item", 
                "changed": false, 
                "item": "latest", 
                "skip_reason": "Conditional result was False", 
                "skipped": true
            }
        ]
    }
}

TASK [Logging to Dockerhub....] ***************************************************************************************************************************
skipping: [172.31.14.65]

TASK [Pushing the latest version changed Dockerimage to Hub] **********************************************************************************************
skipping: [172.31.14.65] => (item=8db1708be153c3b60885c1cc6ce41832ff4e2ad7) 
skipping: [172.31.14.65] => (item=latest) 

TASK [Logout from Hub] ************************************************************************************************************************************
skipping: [172.31.14.65]

TASK [removing the created local images] ******************************************************************************************************************
skipping: [172.31.14.65] => (item=8db1708be153c3b60885c1cc6ce41832ff4e2ad7) 
skipping: [172.31.14.65] => (item=latest) 

PLAY [Deploying version chnaged application on to docker container of dev-server] *************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.8.225 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.8.225]

TASK [Installing Docker on Amazon Linux dev-server] *******************************************************************************************************
ok: [172.31.8.225]

TASK [python docker connector/module] *********************************************************************************************************************
ok: [172.31.8.225]

TASK [Appending ec2-user to docker group] *****************************************************************************************************************
ok: [172.31.8.225]

TASK [Restarting/enabling docker] *************************************************************************************************************************
skipping: [172.31.8.225]

TASK [Creating container at dev-server] *******************************************************************************************************************
ok: [172.31.8.225]

TASK [Debug container] ************************************************************************************************************************************
ok: [172.31.8.225] => {
    "msg": "Container Hostname:6a1420813817 Changed:False created at:2022-07-02T11:20:30.830521237Z Restarting state:False"
}

TASK [Debug git_status from host amazon-image-server] *****************************************************************************************************
ok: [172.31.8.225] => {
    "hostvars['172.31.14.65']['git_status']": {
        "after": "8db1708be153c3b60885c1cc6ce41832ff4e2ad7", 
        "before": "8db1708be153c3b60885c1cc6ce41832ff4e2ad7", 
        "changed": false, 
        "failed": false, 
        "remote_url_changed": false
    }
}

PLAY RECAP ************************************************************************************************************************************************
172.31.14.65               : ok=7    changed=0    unreachable=0    failed=0    skipped=6    rescued=0    ignored=0   
172.31.8.225               : ok=7    changed=0    unreachable=0    failed=0    skipped=1    rescued=0    ignored=0 

## After git version change

In [ ]:
# ansible-playbook -i hosts docker-image.yml 
[WARNING]: Invalid characters were found in group names but not replaced, use -vvvv to see details

PLAY [Creating docker Image on Amazon Linux] **************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.14.65 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.14.65]

TASK [Installing Docker,pip,git on Amazon Linux] **********************************************************************************************************
ok: [172.31.14.65]

TASK [Installing python-docker connector/module] **********************************************************************************************************
ok: [172.31.14.65]

TASK [Adding ec2-user to docker group] ********************************************************************************************************************
ok: [172.31.14.65]

TASK [Restarting/enabling docker] *************************************************************************************************************************
skipping: [172.31.14.65]

TASK [cloning Git Docker Files] ***************************************************************************************************************************
changed: [172.31.14.65]

TASK [Debug git] ******************************************************************************************************************************************
ok: [172.31.14.65] => {
    "git_status": {
        "after": "cd5e2f8ca4ac7ad8fa3d82ab0b9dc2cf47c95cad", 
        "before": "8db1708be153c3b60885c1cc6ce41832ff4e2ad7", 
        "changed": true, 
        "failed": false, 
        "remote_url_changed": false
    }
}

TASK [Building Docker Image] ******************************************************************************************************************************
changed: [172.31.14.65] => (item=cd5e2f8ca4ac7ad8fa3d82ab0b9dc2cf47c95cad)
changed: [172.31.14.65] => (item=latest)

TASK [Debug image_build_status] ***************************************************************************************************************************
ok: [172.31.14.65] => {
    "image_build_status": {
        "changed": true, 
        "msg": "All items completed", 
        "results": [
            {
                "actions": [
                    "Built image jinumona/devops-flaskapp:cd5e2f8ca4ac7ad8fa3d82ab0b9dc2cf47c95cad from /home/ec2-user/image"
                ], 
                "ansible_loop_var": "item", 
                "changed": true, 
                "failed": false, 
                "image": {
                    "Architecture": "amd64", 
                    "Author": "", 
                    "Comment": "", 
                    "Config": {
                        "AttachStderr": false, 
                        "AttachStdin": false, 
                        "AttachStdout": false, 
                        "Cmd": [
                            "python3", 
                            "app.py"
                        ], 
                        "Domainname": "", 
                        "Entrypoint": null, 
                        "Env": [
                            "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
                        ], 
                        "ExposedPorts": {
                            "5000/tcp": {}
                        }, 
                        "Hostname": "", 
                        "Image": "sha256:a33276f362bbf2e54aaabf4a335ddb7f2438be7fcb5aaab4ac3fa40a3cf8d4f2", 
                        "Labels": null, 
                        "OnBuild": null, 
                        "OpenStdin": false, 
                        "StdinOnce": false, 
                        "Tty": false, 
                        "User": "", 
                        "Volumes": null, 
                        "WorkingDir": "/var/flaskapp"
                    }, 
                    "Container": "93a9dd783d343b15b7d113fd3fd0d0c024510d7e58ef08f0054faadf10cbef87", 
                    "ContainerConfig": {
                        "AttachStderr": false, 
                        "AttachStdin": false, 
                        "AttachStdout": false, 
                        "Cmd": [
                            "/bin/sh", 
                            "-c", 
                            "#(nop) ", 
                            "CMD [\"python3\" \"app.py\"]"
                        ], 
                        "Domainname": "", 
                        "Entrypoint": null, 
                        "Env": [
                            "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
                        ], 
                        "ExposedPorts": {
                            "5000/tcp": {}
                        }, 
                        "Hostname": "93a9dd783d34", 
                        "Image": "sha256:a33276f362bbf2e54aaabf4a335ddb7f2438be7fcb5aaab4ac3fa40a3cf8d4f2", 
                        "Labels": {}, 
                        "OnBuild": null, 
                        "OpenStdin": false, 
                        "StdinOnce": false, 
                        "Tty": false, 
                        "User": "", 
                        "Volumes": null, 
                        "WorkingDir": "/var/flaskapp"
                    }, 
                    "Created": "2022-07-02T12:13:21.556754868Z", 
                    "DockerVersion": "20.10.13", 
                    "GraphDriver": {
                        "Data": {
                            "LowerDir": "/var/lib/docker/overlay2/001d816011f708a755cfcab9f2bc0bc8175c5084b17de42b797f7805b44883ff/diff:/var/lib/docker/overlay2/38c0b7078cb7485319651bd5a7d19f1a45d26d70e7503d21eae19663aa4cb45f/diff:/var/lib/docker/overlay2/064ac02ccbd7388f317cee21fb8dbbdb0b0738dac223bf0e6e4b179eaad467c4/diff:/var/lib/docker/overlay2/9cf75159a8fdfad7e38c0e18048eff3aaba003aba6c272961335aa99f8bb6a3b/diff:/var/lib/docker/overlay2/3e25c4b253ab89ec18eb1a2bba6b131af4c834474622b73c5070e2c613a93cdf/diff", 
                            "MergedDir": "/var/lib/docker/overlay2/e4f475e79a1b9086cbd3b66c2fb2b9ad834458f218258b99a6e87180fb48ee48/merged", 
                            "UpperDir": "/var/lib/docker/overlay2/e4f475e79a1b9086cbd3b66c2fb2b9ad834458f218258b99a6e87180fb48ee48/diff", 
                            "WorkDir": "/var/lib/docker/overlay2/e4f475e79a1b9086cbd3b66c2fb2b9ad834458f218258b99a6e87180fb48ee48/work"
                        }, 
                        "Name": "overlay2"
                    }, 
                    "Id": "sha256:bc2976cebfba653ba1bad36bc02ccd6d8982a4d61707a16639972a7c333a8479", 
                    "Metadata": {
                        "LastTagTime": "2022-07-02T12:13:21.576589818Z"
                    }, 
                    "Os": "linux", 
                    "Parent": "sha256:a33276f362bbf2e54aaabf4a335ddb7f2438be7fcb5aaab4ac3fa40a3cf8d4f2", 
                    "RepoDigests": [], 
                    "RepoTags": [
                        "jinumona/devops-flaskapp:cd5e2f8ca4ac7ad8fa3d82ab0b9dc2cf47c95cad"
                    ], 
                    "RootFS": {
                        "Layers": [
                            "sha256:7444ea29e45e927abea1f923bf24cac20deaddea603c4bb1c7f2f5819773d453", 
                            "sha256:d41b10567b03144730b4b004aa89c6505f9fd474e4ed120a6c8dd0ae270cc838", 
                            "sha256:d58853fc6398a5bcf711354eb3ca70ec063357096c25c3427fa53177dfca7fb2", 
                            "sha256:ce07cf50cc6d145d18834754a059c74a60cb41584a697017a68500247a5750df", 
                            "sha256:d0643476e2cfaf22a01c6fef05cb7fd2a4cb97f7bf37cad9d5f583438abd077c", 
                            "sha256:f4647630d8e85d68bd3a17ab3868b68fa11ea932ee2ca8a5f96d56255cbfb28d"
                        ], 
                        "Type": "layers"
                    }, 
                    "Size": 62788887, 
                    "VirtualSize": 62788887
                }, 
                "invocation": {
                    "module_args": {
                        "api_version": "auto", 
                        "archive_path": null, 
                        "build": {
                            "args": null, 
                            "cache_from": null, 
                            "container_limits": null, 
                            "dockerfile": null, 
                            "etc_hosts": null, 
                            "http_timeout": null, 
                            "network": null, 
                            "nocache": false, 
                            "path": "/home/ec2-user/image", 
                            "platform": null, 
                            "pull": false, 
                            "rm": true, 
                            "target": null, 
                            "use_config_proxy": null
                        }, 
                        "ca_cert": null, 
                        "client_cert": null, 
                        "client_key": null, 
                        "debug": false, 
                        "docker_host": "unix://var/run/docker.sock", 
                        "force_absent": false, 
                        "force_source": true, 
                        "force_tag": true, 
                        "load_path": null, 
                        "name": "jinumona/devops-flaskapp", 
                        "pull": null, 
                        "push": false, 
                        "repository": null, 
                        "source": "build", 
                        "ssl_version": null, 
                        "state": "present", 
                        "tag": "cd5e2f8ca4ac7ad8fa3d82ab0b9dc2cf47c95cad", 
                        "timeout": 60, 
                        "tls": false, 
                        "tls_hostname": null, 
                        "use_ssh_client": false, 
                        "validate_certs": false
                    }
                }, 
                "item": "cd5e2f8ca4ac7ad8fa3d82ab0b9dc2cf47c95cad", 
                "stdout": "Step 1/9 : FROM alpine:3.8\n\n ---> c8bccc0af957\nStep 2/9 : RUN mkdir /var/flaskapp\n\n ---> Running in f0cde07791e8\nRemoving intermediate container f0cde07791e8\n ---> 44f926a03c76\nStep 3/9 : WORKDIR /var/flaskapp\n\n ---> Running in c15bcb70b578\nRemoving intermediate container c15bcb70b578\n ---> 573bc6f03978\nStep 4/9 : COPY .  .\n\n ---> 966358a84b8b\nStep 5/9 : RUN apk update\n\n ---> Running in ccca18e45c3d\nfetch http://dl-cdn.alpinelinux.org/alpine/v3.8/main/x86_64/APKINDEX.tar.gz\nfetch http://dl-cdn.alpinelinux.org/alpine/v3.8/community/x86_64/APKINDEX.tar.gz\nv3.8.5-67-gf94de196ca [http://dl-cdn.alpinelinux.org/alpine/v3.8/main]\nv3.8.5-66-gccbd6a8ae7 [http://dl-cdn.alpinelinux.org/alpine/v3.8/community]\nOK: 9564 distinct packages available\nRemoving intermediate container ccca18e45c3d\n ---> e25b2e858fe5\nStep 6/9 : RUN apk add python3\n\n ---> Running in 2318caaacc1b\n(1/11) Installing libbz2 (1.0.6-r7)\n(2/11) Installing expat (2.2.8-r0)\n(3/11) Installing libffi (3.2.1-r4)\n(4/11) Installing gdbm (1.13-r1)\n(5/11) Installing xz-libs (5.2.4-r0)\n(6/11) Installing ncurses-terminfo-base (6.1_p20180818-r1)\n(7/11) Installing ncurses-terminfo (6.1_p20180818-r1)\n(8/11) Installing ncurses-libs (6.1_p20180818-r1)\n(9/11) Installing readline (7.0.003-r0)\n(10/11) Installing sqlite-libs (3.25.3-r4)\n(11/11) Installing python3 (3.6.9-r1)\nExecuting busybox-1.28.4-r3.trigger\nOK: 67 MiB in 24 packages\nRemoving intermediate container 2318caaacc1b\n ---> b905a853f468\nStep 7/9 : RUN pip3 install -r requirement.txt\n\n ---> Running in d0df86e1ffad\nCollecting flask (from -r requirement.txt (line 1))\n  Downloading https://files.pythonhosted.org/packages/cd/77/59df23681f4fd19b7cbbb5e92484d46ad587554f5d490f33ef907e456132/Flask-2.0.3-py3-none-any.whl (95kB)\nCollecting itsdangerous>=2.0 (from flask->-r requirement.txt (line 1))\n  Downloading https://files.pythonhosted.org/packages/9c/96/26f935afba9cd6140216da5add223a0c465b99d0f112b68a4ca426441019/itsdangerous-2.0.1-py3-none-any.whl\nCollecting click>=7.1.2 (from flask->-r requirement.txt (line 1))\n  Downloading https://files.pythonhosted.org/packages/4a/a8/0b2ced25639fb20cc1c9784de90a8c25f9504a7f18cd8b5397bd61696d7d/click-8.0.4-py3-none-any.whl (97kB)\nCollecting Werkzeug>=2.0 (from flask->-r requirement.txt (line 1))\n  Downloading https://files.pythonhosted.org/packages/f4/f3/22afbdb20cc4654b10c98043414a14057cd27fdba9d4ae61cea596000ba2/Werkzeug-2.0.3-py3-none-any.whl (289kB)\nCollecting Jinja2>=3.0 (from flask->-r requirement.txt (line 1))\n  Downloading https://files.pythonhosted.org/packages/20/9a/e5d9ec41927401e41aea8af6d16e78b5e612bca4699d417f646a9610a076/Jinja2-3.0.3-py3-none-any.whl (133kB)\nCollecting importlib-metadata; python_version < \"3.8\" (from click>=7.1.2->flask->-r requirement.txt (line 1))\n  Downloading https://files.pythonhosted.org/packages/a0/a1/b153a0a4caf7a7e3f15c2cd56c7702e2cf3d89b1b359d1f1c5e59d68f4ce/importlib_metadata-4.8.3-py3-none-any.whl\nCollecting dataclasses; python_version < \"3.7\" (from Werkzeug>=2.0->flask->-r requirement.txt (line 1))\n  Downloading https://files.pythonhosted.org/packages/fe/ca/75fac5856ab5cfa51bbbcefa250182e50441074fdc3f803f6e76451fab43/dataclasses-0.8-py3-none-any.whl\nCollecting MarkupSafe>=2.0 (from Jinja2>=3.0->flask->-r requirement.txt (line 1))\n  Downloading https://files.pythonhosted.org/packages/bf/10/ff66fea6d1788c458663a84d88787bae15d45daa16f6b3ef33322a51fc7e/MarkupSafe-2.0.1.tar.gz\nCollecting typing-extensions>=3.6.4; python_version < \"3.8\" (from importlib-metadata; python_version < \"3.8\"->click>=7.1.2->flask->-r requirement.txt (line 1))\n  Downloading https://files.pythonhosted.org/packages/45/6b/44f7f8f1e110027cf88956b59f2fad776cca7e1704396d043f89effd3a0e/typing_extensions-4.1.1-py3-none-any.whl\nCollecting zipp>=0.5 (from importlib-metadata; python_version < \"3.8\"->click>=7.1.2->flask->-r requirement.txt (line 1))\n  Downloading https://files.pythonhosted.org/packages/bd/df/d4a4974a3e3957fd1c1fa3082366d7fff6e428ddb55f074bf64876f8e8ad/zipp-3.6.0-py3-none-any.whl\nInstalling collected packages: itsdangerous, typing-extensions, zipp, importlib-metadata, click, dataclasses, Werkzeug, MarkupSafe, Jinja2, flask\n  Running setup.py install for MarkupSafe: started\n    Running setup.py install for MarkupSafe: finished with status 'done'\nSuccessfully installed Jinja2-3.0.3 MarkupSafe-2.0.1 Werkzeug-2.0.3 click-8.0.4 dataclasses-0.8 flask-2.0.3 importlib-metadata-4.8.3 itsdangerous-2.0.1 typing-extensions-4.1.1 zipp-3.6.0\n\u001b[91mYou are using pip version 18.1, however version 21.3.1 is available.\nYou should consider upgrading via the 'pip install --upgrade pip' command.\n\u001b[0m\nRemoving intermediate container d0df86e1ffad\n ---> 8c6e9ada790d\nStep 8/9 : EXPOSE 5000\n\n ---> Running in c58f4a9361de\nRemoving intermediate container c58f4a9361de\n ---> a33276f362bb\nStep 9/9 : CMD [\"python3\",\"app.py\"]\n\n ---> Running in 93a9dd783d34\nRemoving intermediate container 93a9dd783d34\n ---> bc2976cebfba\nSuccessfully built bc2976cebfba\nSuccessfully tagged jinumona/devops-flaskapp:cd5e2f8ca4ac7ad8fa3d82ab0b9dc2cf47c95cad", 
                "stdout_lines": [
                    "Step 1/9 : FROM alpine:3.8", 
                    "", 
                    " ---> c8bccc0af957", 
                    "Step 2/9 : RUN mkdir /var/flaskapp", 
                    "", 
                    " ---> Running in f0cde07791e8", 
                    "Removing intermediate container f0cde07791e8", 
                    " ---> 44f926a03c76", 
                    "Step 3/9 : WORKDIR /var/flaskapp", 
                    "", 
                    " ---> Running in c15bcb70b578", 
                    "Removing intermediate container c15bcb70b578", 
                    " ---> 573bc6f03978", 
                    "Step 4/9 : COPY .  .", 
                    "", 
                    " ---> 966358a84b8b", 
                    "Step 5/9 : RUN apk update", 
                    "", 
                    " ---> Running in ccca18e45c3d", 
                    "fetch http://dl-cdn.alpinelinux.org/alpine/v3.8/main/x86_64/APKINDEX.tar.gz", 
                    "fetch http://dl-cdn.alpinelinux.org/alpine/v3.8/community/x86_64/APKINDEX.tar.gz", 
                    "v3.8.5-67-gf94de196ca [http://dl-cdn.alpinelinux.org/alpine/v3.8/main]", 
                    "v3.8.5-66-gccbd6a8ae7 [http://dl-cdn.alpinelinux.org/alpine/v3.8/community]", 
                    "OK: 9564 distinct packages available", 
                    "Removing intermediate container ccca18e45c3d", 
                    " ---> e25b2e858fe5", 
                    "Step 6/9 : RUN apk add python3", 
                    "", 
                    " ---> Running in 2318caaacc1b", 
                    "(1/11) Installing libbz2 (1.0.6-r7)", 
                    "(2/11) Installing expat (2.2.8-r0)", 
                    "(3/11) Installing libffi (3.2.1-r4)", 
                    "(4/11) Installing gdbm (1.13-r1)", 
                    "(5/11) Installing xz-libs (5.2.4-r0)", 
                    "(6/11) Installing ncurses-terminfo-base (6.1_p20180818-r1)", 
                    "(7/11) Installing ncurses-terminfo (6.1_p20180818-r1)", 
                    "(8/11) Installing ncurses-libs (6.1_p20180818-r1)", 
                    "(9/11) Installing readline (7.0.003-r0)", 
                    "(10/11) Installing sqlite-libs (3.25.3-r4)", 
                    "(11/11) Installing python3 (3.6.9-r1)", 
                    "Executing busybox-1.28.4-r3.trigger", 
                    "OK: 67 MiB in 24 packages", 
                    "Removing intermediate container 2318caaacc1b", 
                    " ---> b905a853f468", 
                    "Step 7/9 : RUN pip3 install -r requirement.txt", 
                    "", 
                    " ---> Running in d0df86e1ffad", 
                    "Collecting flask (from -r requirement.txt (line 1))", 
                    "  Downloading https://files.pythonhosted.org/packages/cd/77/59df23681f4fd19b7cbbb5e92484d46ad587554f5d490f33ef907e456132/Flask-2.0.3-py3-none-any.whl (95kB)", 
                    "Collecting itsdangerous>=2.0 (from flask->-r requirement.txt (line 1))", 
                    "  Downloading https://files.pythonhosted.org/packages/9c/96/26f935afba9cd6140216da5add223a0c465b99d0f112b68a4ca426441019/itsdangerous-2.0.1-py3-none-any.whl", 
                    "Collecting click>=7.1.2 (from flask->-r requirement.txt (line 1))", 
                    "  Downloading https://files.pythonhosted.org/packages/4a/a8/0b2ced25639fb20cc1c9784de90a8c25f9504a7f18cd8b5397bd61696d7d/click-8.0.4-py3-none-any.whl (97kB)", 
                    "Collecting Werkzeug>=2.0 (from flask->-r requirement.txt (line 1))", 
                    "  Downloading https://files.pythonhosted.org/packages/f4/f3/22afbdb20cc4654b10c98043414a14057cd27fdba9d4ae61cea596000ba2/Werkzeug-2.0.3-py3-none-any.whl (289kB)", 
                    "Collecting Jinja2>=3.0 (from flask->-r requirement.txt (line 1))", 
                    "  Downloading https://files.pythonhosted.org/packages/20/9a/e5d9ec41927401e41aea8af6d16e78b5e612bca4699d417f646a9610a076/Jinja2-3.0.3-py3-none-any.whl (133kB)", 
                    "Collecting importlib-metadata; python_version < \"3.8\" (from click>=7.1.2->flask->-r requirement.txt (line 1))", 
                    "  Downloading https://files.pythonhosted.org/packages/a0/a1/b153a0a4caf7a7e3f15c2cd56c7702e2cf3d89b1b359d1f1c5e59d68f4ce/importlib_metadata-4.8.3-py3-none-any.whl", 
                    "Collecting dataclasses; python_version < \"3.7\" (from Werkzeug>=2.0->flask->-r requirement.txt (line 1))", 
                    "  Downloading https://files.pythonhosted.org/packages/fe/ca/75fac5856ab5cfa51bbbcefa250182e50441074fdc3f803f6e76451fab43/dataclasses-0.8-py3-none-any.whl", 
                    "Collecting MarkupSafe>=2.0 (from Jinja2>=3.0->flask->-r requirement.txt (line 1))", 
                    "  Downloading https://files.pythonhosted.org/packages/bf/10/ff66fea6d1788c458663a84d88787bae15d45daa16f6b3ef33322a51fc7e/MarkupSafe-2.0.1.tar.gz", 
                    "Collecting typing-extensions>=3.6.4; python_version < \"3.8\" (from importlib-metadata; python_version < \"3.8\"->click>=7.1.2->flask->-r requirement.txt (line 1))", 
                    "  Downloading https://files.pythonhosted.org/packages/45/6b/44f7f8f1e110027cf88956b59f2fad776cca7e1704396d043f89effd3a0e/typing_extensions-4.1.1-py3-none-any.whl", 
                    "Collecting zipp>=0.5 (from importlib-metadata; python_version < \"3.8\"->click>=7.1.2->flask->-r requirement.txt (line 1))", 
                    "  Downloading https://files.pythonhosted.org/packages/bd/df/d4a4974a3e3957fd1c1fa3082366d7fff6e428ddb55f074bf64876f8e8ad/zipp-3.6.0-py3-none-any.whl", 
                    "Installing collected packages: itsdangerous, typing-extensions, zipp, importlib-metadata, click, dataclasses, Werkzeug, MarkupSafe, Jinja2, flask", 
                    "  Running setup.py install for MarkupSafe: started", 
                    "    Running setup.py install for MarkupSafe: finished with status 'done'", 
                    "Successfully installed Jinja2-3.0.3 MarkupSafe-2.0.1 Werkzeug-2.0.3 click-8.0.4 dataclasses-0.8 flask-2.0.3 importlib-metadata-4.8.3 itsdangerous-2.0.1 typing-extensions-4.1.1 zipp-3.6.0", 
                    "\u001b[91mYou are using pip version 18.1, however version 21.3.1 is available.", 
                    "You should consider upgrading via the 'pip install --upgrade pip' command.", 
                    "\u001b[0m", 
                    "Removing intermediate container d0df86e1ffad", 
                    " ---> 8c6e9ada790d", 
                    "Step 8/9 : EXPOSE 5000", 
                    "", 
                    " ---> Running in c58f4a9361de", 
                    "Removing intermediate container c58f4a9361de", 
                    " ---> a33276f362bb", 
                    "Step 9/9 : CMD [\"python3\",\"app.py\"]", 
                    "", 
                    " ---> Running in 93a9dd783d34", 
                    "Removing intermediate container 93a9dd783d34", 
                    " ---> bc2976cebfba", 
                    "Successfully built bc2976cebfba", 
                    "Successfully tagged jinumona/devops-flaskapp:cd5e2f8ca4ac7ad8fa3d82ab0b9dc2cf47c95cad"
                ]
            }, 
            {
                "actions": [
                    "Built image jinumona/devops-flaskapp:latest from /home/ec2-user/image"
                ], 
                "ansible_loop_var": "item", 
                "changed": true, 
                "failed": false, 
                "image": {
                    "Architecture": "amd64", 
                    "Author": "", 
                    "Comment": "", 
                    "Config": {
                        "AttachStderr": false, 
                        "AttachStdin": false, 
                        "AttachStdout": false, 
                        "Cmd": [
                            "python3", 
                            "app.py"
                        ], 
                        "Domainname": "", 
                        "Entrypoint": null, 
                        "Env": [
                            "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
                        ], 
                        "ExposedPorts": {
                            "5000/tcp": {}
                        }, 
                        "Hostname": "", 
                        "Image": "sha256:a33276f362bbf2e54aaabf4a335ddb7f2438be7fcb5aaab4ac3fa40a3cf8d4f2", 
                        "Labels": null, 
                        "OnBuild": null, 
                        "OpenStdin": false, 
                        "StdinOnce": false, 
                        "Tty": false, 
                        "User": "", 
                        "Volumes": null, 
                        "WorkingDir": "/var/flaskapp"
                    }, 
                    "Container": "93a9dd783d343b15b7d113fd3fd0d0c024510d7e58ef08f0054faadf10cbef87", 
                    "ContainerConfig": {
                        "AttachStderr": false, 
                        "AttachStdin": false, 
                        "AttachStdout": false, 
                        "Cmd": [
                            "/bin/sh", 
                            "-c", 
                            "#(nop) ", 
                            "CMD [\"python3\" \"app.py\"]"
                        ], 
                        "Domainname": "", 
                        "Entrypoint": null, 
                        "Env": [
                            "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
                        ], 
                        "ExposedPorts": {
                            "5000/tcp": {}
                        }, 
                        "Hostname": "93a9dd783d34", 
                        "Image": "sha256:a33276f362bbf2e54aaabf4a335ddb7f2438be7fcb5aaab4ac3fa40a3cf8d4f2", 
                        "Labels": {}, 
                        "OnBuild": null, 
                        "OpenStdin": false, 
                        "StdinOnce": false, 
                        "Tty": false, 
                        "User": "", 
                        "Volumes": null, 
                        "WorkingDir": "/var/flaskapp"
                    }, 
                    "Created": "2022-07-02T12:13:21.556754868Z", 
                    "DockerVersion": "20.10.13", 
                    "GraphDriver": {
                        "Data": {
                            "LowerDir": "/var/lib/docker/overlay2/001d816011f708a755cfcab9f2bc0bc8175c5084b17de42b797f7805b44883ff/diff:/var/lib/docker/overlay2/38c0b7078cb7485319651bd5a7d19f1a45d26d70e7503d21eae19663aa4cb45f/diff:/var/lib/docker/overlay2/064ac02ccbd7388f317cee21fb8dbbdb0b0738dac223bf0e6e4b179eaad467c4/diff:/var/lib/docker/overlay2/9cf75159a8fdfad7e38c0e18048eff3aaba003aba6c272961335aa99f8bb6a3b/diff:/var/lib/docker/overlay2/3e25c4b253ab89ec18eb1a2bba6b131af4c834474622b73c5070e2c613a93cdf/diff", 
                            "MergedDir": "/var/lib/docker/overlay2/e4f475e79a1b9086cbd3b66c2fb2b9ad834458f218258b99a6e87180fb48ee48/merged", 
                            "UpperDir": "/var/lib/docker/overlay2/e4f475e79a1b9086cbd3b66c2fb2b9ad834458f218258b99a6e87180fb48ee48/diff", 
                            "WorkDir": "/var/lib/docker/overlay2/e4f475e79a1b9086cbd3b66c2fb2b9ad834458f218258b99a6e87180fb48ee48/work"
                        }, 
                        "Name": "overlay2"
                    }, 
                    "Id": "sha256:bc2976cebfba653ba1bad36bc02ccd6d8982a4d61707a16639972a7c333a8479", 
                    "Metadata": {
                        "LastTagTime": "2022-07-02T12:13:22.115935304Z"
                    }, 
                    "Os": "linux", 
                    "Parent": "sha256:a33276f362bbf2e54aaabf4a335ddb7f2438be7fcb5aaab4ac3fa40a3cf8d4f2", 
                    "RepoDigests": [], 
                    "RepoTags": [
                        "jinumona/devops-flaskapp:cd5e2f8ca4ac7ad8fa3d82ab0b9dc2cf47c95cad", 
                        "jinumona/devops-flaskapp:latest"
                    ], 
                    "RootFS": {
                        "Layers": [
                            "sha256:7444ea29e45e927abea1f923bf24cac20deaddea603c4bb1c7f2f5819773d453", 
                            "sha256:d41b10567b03144730b4b004aa89c6505f9fd474e4ed120a6c8dd0ae270cc838", 
                            "sha256:d58853fc6398a5bcf711354eb3ca70ec063357096c25c3427fa53177dfca7fb2", 
                            "sha256:ce07cf50cc6d145d18834754a059c74a60cb41584a697017a68500247a5750df", 
                            "sha256:d0643476e2cfaf22a01c6fef05cb7fd2a4cb97f7bf37cad9d5f583438abd077c", 
                            "sha256:f4647630d8e85d68bd3a17ab3868b68fa11ea932ee2ca8a5f96d56255cbfb28d"
                        ], 
                        "Type": "layers"
                    }, 
                    "Size": 62788887, 
                    "VirtualSize": 62788887
                }, 
                "invocation": {
                    "module_args": {
                        "api_version": "auto", 
                        "archive_path": null, 
                        "build": {
                            "args": null, 
                            "cache_from": null, 
                            "container_limits": null, 
                            "dockerfile": null, 
                            "etc_hosts": null, 
                            "http_timeout": null, 
                            "network": null, 
                            "nocache": false, 
                            "path": "/home/ec2-user/image", 
                            "platform": null, 
                            "pull": false, 
                            "rm": true, 
                            "target": null, 
                            "use_config_proxy": null
                        }, 
                        "ca_cert": null, 
                        "client_cert": null, 
                        "client_key": null, 
                        "debug": false, 
                        "docker_host": "unix://var/run/docker.sock", 
                        "force_absent": false, 
                        "force_source": true, 
                        "force_tag": true, 
                        "load_path": null, 
                        "name": "jinumona/devops-flaskapp", 
                        "pull": null, 
                        "push": false, 
                        "repository": null, 
                        "source": "build", 
                        "ssl_version": null, 
                        "state": "present", 
                        "tag": "latest", 
                        "timeout": 60, 
                        "tls": false, 
                        "tls_hostname": null, 
                        "use_ssh_client": false, 
                        "validate_certs": false
                    }
                }, 
                "item": "latest", 
                "stdout": "Step 1/9 : FROM alpine:3.8\n\n ---> c8bccc0af957\nStep 2/9 : RUN mkdir /var/flaskapp\n\n ---> Using cache\n ---> 44f926a03c76\nStep 3/9 : WORKDIR /var/flaskapp\n\n ---> Using cache\n ---> 573bc6f03978\nStep 4/9 : COPY .  .\n\n ---> Using cache\n ---> 966358a84b8b\nStep 5/9 : RUN apk update\n\n ---> Using cache\n ---> e25b2e858fe5\nStep 6/9 : RUN apk add python3\n\n ---> Using cache\n ---> b905a853f468\nStep 7/9 : RUN pip3 install -r requirement.txt\n\n ---> Using cache\n ---> 8c6e9ada790d\nStep 8/9 : EXPOSE 5000\n\n ---> Using cache\n ---> a33276f362bb\nStep 9/9 : CMD [\"python3\",\"app.py\"]\n\n ---> Using cache\n ---> bc2976cebfba\nSuccessfully built bc2976cebfba\nSuccessfully tagged jinumona/devops-flaskapp:latest", 
                "stdout_lines": [
                    "Step 1/9 : FROM alpine:3.8", 
                    "", 
                    " ---> c8bccc0af957", 
                    "Step 2/9 : RUN mkdir /var/flaskapp", 
                    "", 
                    " ---> Using cache", 
                    " ---> 44f926a03c76", 
                    "Step 3/9 : WORKDIR /var/flaskapp", 
                    "", 
                    " ---> Using cache", 
                    " ---> 573bc6f03978", 
                    "Step 4/9 : COPY .  .", 
                    "", 
                    " ---> Using cache", 
                    " ---> 966358a84b8b", 
                    "Step 5/9 : RUN apk update", 
                    "", 
                    " ---> Using cache", 
                    " ---> e25b2e858fe5", 
                    "Step 6/9 : RUN apk add python3", 
                    "", 
                    " ---> Using cache", 
                    " ---> b905a853f468", 
                    "Step 7/9 : RUN pip3 install -r requirement.txt", 
                    "", 
                    " ---> Using cache", 
                    " ---> 8c6e9ada790d", 
                    "Step 8/9 : EXPOSE 5000", 
                    "", 
                    " ---> Using cache", 
                    " ---> a33276f362bb", 
                    "Step 9/9 : CMD [\"python3\",\"app.py\"]", 
                    "", 
                    " ---> Using cache", 
                    " ---> bc2976cebfba", 
                    "Successfully built bc2976cebfba", 
                    "Successfully tagged jinumona/devops-flaskapp:latest"
                ]
            }
        ]
    }
}

TASK [Logging to Dockerhub....] ***************************************************************************************************************************
changed: [172.31.14.65]

TASK [Pushing the latest version changed Dockerimage to Hub] **********************************************************************************************
changed: [172.31.14.65] => (item=cd5e2f8ca4ac7ad8fa3d82ab0b9dc2cf47c95cad)
ok: [172.31.14.65] => (item=latest)

TASK [Logout from Hub] ************************************************************************************************************************************
changed: [172.31.14.65]

TASK [removing the created local images] ******************************************************************************************************************
changed: [172.31.14.65] => (item=cd5e2f8ca4ac7ad8fa3d82ab0b9dc2cf47c95cad)
changed: [172.31.14.65] => (item=latest)

PLAY [Deploying version chnaged application on to docker container of dev-server] *************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.8.225 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.8.225]

TASK [Installing Docker on Amazon Linux dev-server] *******************************************************************************************************
ok: [172.31.8.225]

TASK [python docker connector/module] *********************************************************************************************************************
ok: [172.31.8.225]

TASK [Appending ec2-user to docker group] *****************************************************************************************************************
ok: [172.31.8.225]

TASK [Restarting/enabling docker] *************************************************************************************************************************
skipping: [172.31.8.225]

TASK [Creating container at dev-server] *******************************************************************************************************************
changed: [172.31.8.225]

TASK [Debug container] ************************************************************************************************************************************
ok: [172.31.8.225] => {
    "msg": "Container Hostname:426fa3129829 Changed:True created at:2022-07-02T12:14:15.938582105Z Restarting state:False"
}

TASK [Debug git_status from host amazon-image-server] *****************************************************************************************************
ok: [172.31.8.225] => {
    "hostvars['172.31.14.65']['git_status']": {
        "after": "cd5e2f8ca4ac7ad8fa3d82ab0b9dc2cf47c95cad", 
        "before": "8db1708be153c3b60885c1cc6ce41832ff4e2ad7", 
        "changed": true, 
        "failed": false, 
        "remote_url_changed": false
    }
}

PLAY RECAP ************************************************************************************************************************************************
172.31.14.65               : ok=12   changed=6    unreachable=0    failed=0    skipped=1    rescued=0    ignored=0   
172.31.8.225               : ok=7    changed=1    unreachable=0    failed=0    skipped=1    rescued=0    ignored=0

### On dev server

In [ ]:
]# docker container ls -a
CONTAINER ID   IMAGE                             COMMAND            CREATED          STATUS          PORTS                    NAMES
6a1420813817   jinumona/devops-flaskapp:latest   "python3 app.py"   51 minutes ago   Up 51 minutes   0.0.0.0:8081->5000/tcp   myflaskcntr8081
[root@ip-172-31-8-225 ec2-user]# docker image ls
REPOSITORY                 TAG       IMAGE ID       CREATED             SIZE
jinumona/devops-flaskapp   latest    49ae7760de60   About an hour ago   62.8MB
jinumona/devops-flaskapp   <none>    130623177fe3   13 hours ago        62.8MB
jinumona/devops-flaskapp   <none>    a6ad91bcc2ca   15 hours ago        62.8MB
[root@ip-172-31-8-225 ec2-user]# after git version chnaged on this dev-server
bash: after: command not found
[root@ip-172-31-8-225 ec2-user]# docker image ls
REPOSITORY                 TAG       IMAGE ID       CREATED              SIZE
jinumona/devops-flaskapp   latest    bc2976cebfba   About a minute ago   62.8MB
jinumona/devops-flaskapp   <none>    49ae7760de60   2 hours ago          62.8MB
jinumona/devops-flaskapp   <none>    130623177fe3   13 hours ago         62.8MB
jinumona/devops-flaskapp   <none>    a6ad91bcc2ca   15 hours ago         62.8MB
[root@ip-172-31-8-225 ec2-user]# docker container ls -a
CONTAINER ID   IMAGE                             COMMAND            CREATED              STATUS              PORTS                    NAMES
426fa3129829   jinumona/devops-flaskapp:latest   "python3 app.py"   About a minute ago   Up About a minute   0.0.0.0:8081->5000/tcp   myflaskcntr8081
